In [1]:
# import opencv
import cv2
import matplotlib.pyplot as plt
import numpy as np

# load image
img = cv2.imread('./grid1200.png')
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Initialize sensible parameters for cv2.SimpleBlobDetector
params = cv2.SimpleBlobDetector_Params()

# Filter by color
params.filterByColor = True
params.blobColor = 255

# Filter by circularity
params.filterByCircularity = True
params.minCircularity = 0.75

# Filter by convexity
params.filterByConvexity = True
params.minConvexity = 0.75

detector = cv2.SimpleBlobDetector_create(params)

keypoints = detector.detect(grey)

img_with_keypoints = cv2.drawKeypoints(img, keypoints, np.array([]), (255,0,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# images with a lot of blobs are likely to fail in the findCirclesGrid function,
# so crop the image to near the centroid of the blob mass


# locate the pixel grid tolerant of camera distortion
ret, corners = cv2.findCirclesGrid(grey, (20, 20), None, flags=(cv2.CALIB_CB_SYMMETRIC_GRID ), blobDetector=detector )

img_with_corners = cv2.drawChessboardCorners(img.copy(), (20,20), corners, ret)

np_corners = np.array(corners).reshape(20, 20, 2)

# find top left and bottom right corners
top_left = np.rint(np_corners[0][0]).astype(int)
top_right = np.rint(np_corners[0][-1]).astype(int)
bottom_left = np.rint(np_corners[-1][0]).astype(int)
bottom_right = np.rint(np_corners[-1][-1]).astype(int)

# two dimensional distances to account for rotation
width = np.linalg.norm(top_left - top_right).astype(int)
height = np.linalg.norm(top_left - bottom_left).astype(int)

# prepare ideal object points to map corners to
objp = np.zeros((20*20,3), np.float32)
# fill with grid from 0 to width, 0 to height with 20 steps
objp[:,:2] = np.mgrid[0:width:20j,:height:20j].T.reshape(-1,2)

# find homography transformation to aligned grid
# both corners and objp are 2d arrays of 20x20 points, so I presume should
# map corners to objp which has a dimension of height x width
H, _ = cv2.findHomography(corners, objp)

# warp image to axis-aligned planar
# width and height clip the output?
grey_with_warp = cv2.warpPerspective(grey, H, (width, height))
img_with_warp_corners = cv2.warpPerspective(img_with_corners, H, (width, height))

ret, grey_with_warp = cv2.threshold(grey_with_warp, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Find contours and find total area
cnts = cv2.findContours(grey_with_warp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
area = 0
for c in cnts:
    area += cv2.contourArea(c)

print(f'fill factor: {area / (width * height) * 100:0.1f}%')

# convert to RGB
img_with_keypoints = cv2.cvtColor(img_with_keypoints, cv2.COLOR_BGR2RGB)
img_with_corners = cv2.cvtColor(img_with_corners, cv2.COLOR_BGR2RGB)
img_with_warp_corners = cv2.cvtColor(img_with_warp_corners, cv2.COLOR_BGR2RGB)
grey_with_warp = cv2.cvtColor(grey_with_warp, cv2.COLOR_GRAY2RGB)

# save images
cv2.imwrite('output_1_keypoints.png', img_with_keypoints)
cv2.imwrite('output_2_corners.png', img_with_corners)
cv2.imwrite('output_3_warp_corners.png', img_with_warp_corners)
cv2.imwrite('output_4_warp_thresh.png', grey_with_warp)

fill factor: 18.1%


True